In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

from sklearn.metrics import confusion_matrix

import itertools

from datetime import datetime

%matplotlib inline

In [ ]:
train_dataset = torchvision.datasets.FashionMNIST(root=".", 
                                                  train=True, 
                                                  transform=transforms.ToTensor(), 
                                                  download=True)
train_dataset.data.shape

In [ ]:
test_dataset = torchvision.datasets.FashionMNIST(root=".", 
                                                 train=False, 
                                                 transform=transforms.ToTensor(), 
                                                 download=True)
test_dataset.data.shape

In [ ]:
number_of_classes = len(set(train_dataset.targets.numpy()))
number_of_classes

In [ ]:
# Define the model
class CNN(nn.Module):
    def __init__(self, number_of_classes):
        super(CNN, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=2), 
            nn.ReLU(), 
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=2), 
            nn.ReLU(), 
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=2), 
            nn.ReLU()
        )

        self.dense_layers = nn.Sequential(
            nn.Dropout(0.2),
            nn.Linear(128 * 2 * 2, 512), 
            nn.ReLU(),
            nn.Dropout(0.2), 
            nn.Linear(512, 200),
            nn.ReLU(),
            nn.Dropout(0.2), 
            nn.Linear(200, 100),
            nn.ReLU(),
            nn.Dropout(0.2), 
            nn.Linear(100, number_of_classes)
        )

    def forward(self, X):
        out = self.conv_layers(X)
        out = out.view(out.size(0), -1)
        out = self.dense_layers(out)
        return out

In [ ]:
model = CNN(number_of_classes)

In [ ]:
model

In [ ]:
# model = nn.Sequential(
#     nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=2), 
#     nn.ReLU(), 
#     nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=2), 
#     nn.ReLU(), 
#     nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=2), 
#     nn.ReLU(), 
#     nn.Flatten()
#     nn.Linear(128 * 2 * 2, 512), 
#     nn.ReLU(), 
#     nn.Dropout(0.2), 
#     nn.Linear(512, 200), 
#     nn.ReLU(),
#     nn.Dropout(0.2),
#     nn.Linear(200, 100), 
#     nn.ReLU(),
#     nn.Dropout(0.2), 
#     nn.Linear(100, 10)        
# )

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
model.to(device)
model

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters())

In [ ]:
batch_size = 128
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                            batch_size=batch_size, 
                            shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

In [ ]:
def batch_gd(model, criterion, optimizer, train_loader, test_loader, epochs):
    train_losses = np.zeros(epochs)
    test_losses = np.zeros(epochs)

    for epoch in range(epochs):
        t0 = datetime.now()
        train_loss = []
        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, targets)
            train_loss.append(loss.item())

            loss.backward()
            optimizer.step()
        train_losses[epoch] = np.mean(train_loss)

        test_loss = []
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, targets)
            test_loss.append(loss.item())
        test_losses[epoch] = np.mean(test_loss)

        dt = datetime.now() - t0

    print(f"Epoch {epoch + 1}/{epochs}, Train Loss: {np.mean(train_loss):.4f}, Test Loss: {np.mean(test_loss):.4f}, Duration: {dt}")

    return train_losses, test_losses

In [ ]:
train_losses, test_losses = batch_gd(model, criterion, optimizer,
                                  train_loader, test_loader, 15)

In [ ]:
plt.plot(train_losses, label="train_loss")
plt.plot(test_losses, label="test_loss")
plt.legend()
plt.plot()

In [ ]:
n_correct = 0
n_total = 0

for inputs, targets in train_loader:
    inputs, targets = inputs.to(device), targets.to(device)

    outputs = model(inputs)
    _, predictions = torch.max(outputs, 1)

    n_correct += (predictions == targets).sum().item()
    n_total += targets.shape[0]

test_acc = (n_correct / n_total) * 100

print(f"Test Accuracy: {test_acc:.4f}%")


n_correct = 0
n_total = 0

for inputs, targets in test_loader:
    inputs, targets = inputs.to(device), targets.to(device)

    outputs = model(inputs)
    _, predictions = torch.max(outputs, 1)

    n_correct += (predictions == targets).sum().item()
    n_total += targets.shape[0]

    train_acc = (n_correct / n_total) * 100
print(f"Train Accuracy: {train_acc:.4f}%")

In [ ]:
def plot_confucsion_matrix(cm, classes, 
                           normalize=False,
                           title="Confusion Matrix", 
                           cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """

    if normalize:
        cm = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis]
        print("normalized confusion matrix")
    else:
        print("Confusion matrix, without normalization")    

    print(cm)

    plt.imshow(cm, interpolation="nearest", cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90)
    plt.yticks(tick_marks, classes)

    fmt = ".2f" if normalize else "d"
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),   
                 horizontalalignment="center",           
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel("True label")
    plt.xlabel("Predicted label")
    plt.show()

In [ ]:
labels = """T-shirt/top
Trouser
Pullover
Dress
Coat
Sandal
Shirt
Sneaker
Bag
Ankle boot""".split("\n")

x_test = test_dataset.data.numpy()
y_test = test_dataset.targets.numpy()
p_test = np.array([])

for inputs, targets in test_loader:
    inputs = inputs.to(device)

    outputs = model(inputs)

    _, predictions = torch.max(outputs, 1)

    p_test = np.concatenate((p_test, predictions.cpu().numpy()))

cm = confusion_matrix(y_test, p_test)
plot_confucsion_matrix(cm, labels)

In [ ]:
misclassifed_idx = np.where(p_test != y_test)[0]
i = np.random.choice(misclassifed_idx)
plt.imshow(x_test[i], cmap="gray")
plt.title("True Label: %s predicted: %s" % (labels[y_test[i]], labels[int(p_test[i])]))
plt.show()